## Regressão Linear

Usando Regressão Linear para prever o consumo de combustível de automóveis.

A variável consumo no dataset1.csv será a variável target (dependente) e as demais variáveis serão candidatas a features (variáveis preditoras ou independetes).

Este é, portanto, um problema de Regressão Linear Múltipla (quando temos mais de 1 variável preditora ou independente).

In [1]:
# Importa o findspark e incializa
import findspark
findspark.init()

In [2]:
# Imports
import numpy as np
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

### Carregando os Dados


In [3]:
# Criando o Spark Context
sc = SparkContext(appName = "Projeto - Prevendo Consumo Commbustível")

23/03/20 14:46:14 WARN Utils: Your hostname, ingo-Vostro-3583 resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlo1)
23/03/20 14:46:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 14:46:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/20 14:46:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Deixando apenas as mensagens de erro
sc.setLogLevel('ERROR')

In [5]:
# Criando o Spark Session
spSession = SparkSession.builder.master('local').getOrCreate()

In [6]:
# Carregando os dados e gerando um RDD
dados = sc.textFile('dados/dataset1.csv')

In [7]:
type(dados)

pyspark.rdd.RDD

In [9]:
# Colocando o RDD em cache. Esse processo otimiza a performance
dados.cache()

dados/dataset1.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [10]:
# Numero de registros
dados.count()

399

In [11]:
# Visualizando as primeiras linhas
dados.take(5)

['consumo,numero_cilindros,capacidade,horsepower,peso,aceleracao,ano,nome',
 '30,4,79,70,2074,19.5,71,peugeot 304',
 '30,4,88,76,2065,14.5,71,fiat 124b',
 '31,4,71,65,1773,19,71,toyota corolla 1200',
 '35,4,72,69,1613,18,71,datsun 1200']

In [12]:
# Removendo a primeira linha do arquivo (cabeçalho)
dados2 = dados.filter(lambda x: 'aceleracao' not in x)
dados2.count()

398

In [14]:
dados2.take(5)

['30,4,79,70,2074,19.5,71,peugeot 304',
 '30,4,88,76,2065,14.5,71,fiat 124b',
 '31,4,71,65,1773,19,71,toyota corolla 1200',
 '35,4,72,69,1613,18,71,datsun 1200',
 '27,4,97,60,1834,19,71,volkswagen model 111']

### Limpeza de Dados

Vamos verificar se há valores ausentes. RDDS são ótimos para processamento, mas ruins para exploração, então convertemos o RDD para DataFrame Spark e então para DataFrame Pandas.

In [18]:
# Converte o RDD para DataFrame Spark
df_spark = dados2.map(lambda x: str(x)).map(lambda w: w.split(',')).toDF()

In [19]:
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [20]:
# Converte DataFrame Spark para DataFrame Pandas
df_pandas = df_spark.toPandas()

In [21]:
type(df_pandas)

pandas.core.frame.DataFrame

In [23]:
df_pandas.head()

,_1,_2,_3,_4,_5,_6,_7,_8
0,30,4,79,70,2074,19.5,71,peugeot 304
1,30,4,88,76,2065,14.5,71,fiat 124b
2,31,4,71,65,1773,19,71,toyota corolla 1200
3,35,4,72,69,1613,18,71,datsun 1200
4,27,4,97,60,1834,19,71,volkswagen model 111


In [24]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _1      398 non-null    object
 1   _2      398 non-null    object
 2   _3      398 non-null    object
 3   _4      398 non-null    object
 4   _5      398 non-null    object
 5   _6      398 non-null    object
 6   _7      398 non-null    object
 7   _8      398 non-null    object
dtypes: object(8)
memory usage: 25.0+ KB


In [26]:
# Tem valores nulos?
df_pandas.isnull().values.any()

False

Não há valor nulo, mas pode ser que exista valor ausente (falta de informação)

In [29]:
# Verifica se há alguma linha com caracter especial "?"
total = np.sum(df_pandas.apply(lambda x: x.str.contains("\?")).values)
total

6

In [30]:
# Verifica quais linhas e colunas têm o caracter especial
np.where(df_pandas.apply(lambda x: x.str.contains("\?")).values)

(array([ 48, 126, 330, 336, 354, 374]), array([3, 3, 3, 3, 3, 3]))

In [31]:
# Visuzliando uma linha
df_pandas.iloc[336]

_1                  23.6
_2                     4
_3                   140
_4                     ?
_5                  2905
_6                  14.3
_7                    80
_8    ford mustang cobra
Name: 336, dtype: object

In [35]:
# Usando um valor padrão para averege HP (que será usado para preencher os valores ausentes)
mediaHP = sc.broadcast(75.0)

In [44]:
# Função para limpeza de dados
def limpaDados(inputStr):
    
    # Variável global
    global mediaHP
    
    # Lista de atributos
    attList = inputStr.split(',')
    
    # Substitui o caracter ? por um valor na coluna de índice 3
    hpValue = attList[3]
    if hpValue == '?':
        hpValue = mediaHP.value
        
    # Cria uma linha usando a função Row, limpando e convertendo os dados de string para float
    linhas = Row(consumo = float(attList[0]),
                 numero_cilindros = float(attList[1]),
                 capacidade = float(attList[2]),
                 horsepower = float(hpValue),
                 peso = float(attList[4]),
                 aceleracao = float(attList[5]),
                 ano = float(attList[6]),
                 nome = attList[7])
    return linhas

In [45]:
# Executa a função no RDD
dados3 = dados2.map(limpaDados)
dados3.cache()
dados3.take(5)

[Row(consumo=30.0, numero_cilindros=4.0, capacidade=79.0, horsepower=70.0, peso=2074.0, aceleracao=19.5, ano=71.0, nome='peugeot 304'),
 Row(consumo=30.0, numero_cilindros=4.0, capacidade=88.0, horsepower=76.0, peso=2065.0, aceleracao=14.5, ano=71.0, nome='fiat 124b'),
 Row(consumo=31.0, numero_cilindros=4.0, capacidade=71.0, horsepower=65.0, peso=1773.0, aceleracao=19.0, ano=71.0, nome='toyota corolla 1200'),
 Row(consumo=35.0, numero_cilindros=4.0, capacidade=72.0, horsepower=69.0, peso=1613.0, aceleracao=18.0, ano=71.0, nome='datsun 1200'),
 Row(consumo=27.0, numero_cilindros=4.0, capacidade=97.0, horsepower=60.0, peso=1834.0, aceleracao=19.0, ano=71.0, nome='volkswagen model 111')]

### Análise Exploratória de Dados


In [47]:
# Cria um Dataframe
df_carros = spSession.createDataFrame(dados3)

In [49]:
# Estatísticas descritivas de duas variáveis (como exemplo)
df_carros.select('consumo', 'numero_cilindros').describe().show()

+-------+-----------------+-----------------+
|summary|          consumo| numero_cilindros|
+-------+-----------------+-----------------+
|  count|              398|              398|
|   mean|23.51457286432161|5.454773869346734|
| stddev|7.815984312565782|1.701004244533212|
|    min|              9.0|              3.0|
|    max|             46.6|              8.0|
+-------+-----------------+-----------------+



In [51]:
# Encontrando a correlação entre a variável target com as variáveis preditoras (exceto o nome)
for i in df_carros.columns:
    if not (isinstance(df_carros.select(i).take(1)[0][0], str)) :
        print('Correlação da variável target com:', i, df_carros.stat.corr('consumo', i))

Correlação da variável target com: consumo 1.0
Correlação da variável target com: numero_cilindros -0.7753962854205546
Correlação da variável target com: capacidade -0.8042028248058979
Correlação da variável target com: horsepower -0.7747041523498721
Correlação da variável target com: peso -0.8317409332443347
Correlação da variável target com: aceleracao 0.42028891210164976
Correlação da variável target com: ano 0.5792671330833098


### Pré-Processamento de Dados

In [53]:
# Convertendo para um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes ou com baixa correlação
def transformaVar(row):
    obj = (row['consumo'], Vectors.dense([row['peso'], row['capacidade'], row['numero_cilindros']]))
    return obj

In [55]:
# Aplica a função no RDD e cria outro RDD
dados4 = dados3.map(transformaVar)

In [56]:
dados4.take(5)

[(30.0, DenseVector([2074.0, 79.0, 4.0])),
 (30.0, DenseVector([2065.0, 88.0, 4.0])),
 (31.0, DenseVector([1773.0, 71.0, 4.0])),
 (35.0, DenseVector([1613.0, 72.0, 4.0])),
 (27.0, DenseVector([1834.0, 97.0, 4.0]))]

In [57]:
# Converte o RDD para DataFrame do Spark
df_carros = spSession.createDataFrame(dados4, ['label', 'features'])

In [58]:
# Visualiza label (y) e atributos (x)
df_carros.select('label', 'features').show(10)

+-----+------------------+
|label|          features|
+-----+------------------+
| 30.0| [2074.0,79.0,4.0]|
| 30.0| [2065.0,88.0,4.0]|
| 31.0| [1773.0,71.0,4.0]|
| 35.0| [1613.0,72.0,4.0]|
| 27.0| [1834.0,97.0,4.0]|
| 26.0| [1955.0,91.0,4.0]|
| 24.0|[2278.0,113.0,4.0]|
| 25.0| [2126.0,97.5,4.0]|
| 23.0| [2254.0,97.0,4.0]|
| 20.0|[2408.0,140.0,4.0]|
+-----+------------------+
only showing top 10 rows



In [59]:
# Divisão em dados de Treino e de Teste com split 70/30
(dados_treino, dados_teste) = df_carros.randomSplit([0.7, 0.3])

In [60]:
dados_treino.count()

279

In [61]:
dados_teste.count()

119

### Machine Learning


In [64]:
# Cria o objeto
linearReg = LinearRegression()

In [65]:
# Treina o objeto com dados e cria o modelo
modelo = linearReg.fit(dados_treino)

In [66]:
print(modelo)

LinearRegressionModel: uid=LinearRegression_1398b9f8301b, numFeatures=3


In [69]:
# Imprimindo os coeficientes (o que o modelo aprendeu)
print('Coeficientes: ' + str(modelo.coefficients))
print('Intercepto: ' + str(modelo.intercept))

Coeficientes: [-0.005326786374572659,-0.01887302259288796,0.025103056259989452]
Intercepto: 42.71457564366255


In [70]:
# Previsões com dados de teste
predictions = modelo.transform(dados_teste)

In [71]:
# Visualiza as previsões
predictions.select('features', 'prediction').show()

+------------------+------------------+
|          features|        prediction|
+------------------+------------------+
|[4382.0,318.0,8.0]|  13.5718010158267|
|[4100.0,350.0,8.0]|14.470018050483777|
|[4502.0,350.0,8.0]| 12.32864992790557|
|[4699.0,350.0,8.0]|11.279273012114754|
|[4746.0,400.0,8.0]| 10.08526292286544|
|[4077.0,318.0,8.0]|15.196470860071361|
|[4129.0,351.0,8.0]| 14.29666822302828|
|[4154.0,351.0,8.0]|14.163498563663964|
|[4257.0,304.0,8.0]|14.501871628948717|
|[4385.0,400.0,8.0]|12.008232804086173|
|[4638.0,302.0,8.0]|12.510112065422305|
|[3730.0,258.0,6.0]|18.127040975101373|
|[3892.0,304.0,8.0]|16.446148655667738|
|[3278.0,250.0,6.0]| 20.68573259715132|
|[3632.0,258.0,6.0]|18.649066039809494|
|[3897.0,250.0,6.0]| 17.38845183129084|
|[4190.0,318.0,8.0]|14.594543999744651|
|[4498.0,318.0,8.0]| 12.95389379637627|
|[3329.0,250.0,6.0]| 20.41406649204811|
|[3907.0,231.0,6.0]| 17.69377139680999|
+------------------+------------------+
only showing top 20 rows



In [73]:
# Coefeficiente de determinação R2
avaliador = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'label', metricName = 'r2')

In [75]:
# Resultado
avaliador.evaluate(predictions)

0.6712444971742968